In [1]:
from algorithms import BPR
import pandas as pd
import numpy as np
import os,sys
from scipy.sparse import *

In [2]:
def load_data(csv_file, shape):
    tp = pd.read_csv(csv_file)
    rows, cols, ratings = np.array(tp['uid']), np.array(tp['sid']), np.array(tp['rating'])
    seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int') ), axis=1)
    data = csr_matrix((ratings, (rows, cols)), dtype=np.float32, shape=shape)
    return data, seq



In [3]:
DATA_DIR = 'data/ml-1m/pro'
uid_fname = 'unique_uid.txt'
sid_fname = 'unique_sid.txt'
rating_fname = 'train.csv'
test_fname ='test.csv'
unique_uid = list()
with open(os.path.join(DATA_DIR, uid_fname), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
unique_sid = list()

with open(os.path.join(DATA_DIR, sid_fname), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())
n_items = len(unique_sid)
n_users = len(unique_uid)

train_data, train_raw = load_data(os.path.join(DATA_DIR, rating_fname),(n_users,n_items))
test_data, test_raw = load_data(os.path.join(DATA_DIR, test_fname),(n_users,n_items))


In [4]:
def bi_mat(mat):
    r_mat = mat.copy()
    r_mat[mat <= 3] = 0
    r_mat[mat > 3] = 1
    r_mat.eliminate_zeros()
    return r_mat

In [6]:
train_data = bi_mat(train_data)
test_data = bi_mat(test_data)

/home/ita/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:274: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using <= is inefficient, try using > instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


MemoryError: 

In [ ]:
print(test_data.nnz / np.prod(test_data.shape))
print(train_data.nnz / np.prod(train_data.shape))

In [ ]:
import importlib
importlib.reload(BPR)
model = BPR.BPR(dtype=np.float32, latent_dim=30, learning_rate=0.03)

In [ ]:
model.train_model(train_data,vad_data=None,n_iter=15)

In [ ]:
u = model.theta
v = model.beta

In [ ]:
import eval
eval.prec_at_k(train_data,test_data,u,v,k=5)